# Introduction

The PatchCamelyon dataset is a set of 327680 colour images of histopathologic scans. These patches are labelled either cancerous, or non-cancerous, based on the presence of metastatic tissue. This project (started in COGS118A at UC San Diego) aims to create deep learning convolutional neural nets to classify these images.

# Initial Tests

Starting with transfer learning, we imported many ImageNet pre-trained models from the PyTorch library. Replacing the last layer with a fully connected layer, we froze all other layers and trained just that singular layer. We tried different image sizes, batch sizes, optimisers, etc. These models yielded a wide variety of results, but none cracked the 80% test accuracy boundary, and seemed to stagnate after 5 or so epochs.

<img src="grid_search_1.png"  width="500">

Models such as ResNet50 are optimised for images of size 224x224 or larger, so images from this dataset, of size 96x96 (out of which only a center crop of 32x32 was supposed to be checked) did not perform well on the model. [Geert Litjens](https://geertlitjens.nl/post/getting-started-with-camelyon/), one of the publishers of this dataset, recommends a 6 layer convolutional, 3 layer dense, 2 layer dropout network, as a relatively shallow network equipped with dealing with small images. Following this strategy without using any transfer learning, we started getting relatively high training accuracies, breaking the 80% boundary. The code for this model is provided below this writeup.

After trying over 50 models and parameter combinations over 3 weeks, we started fine tuning our model.

# Fine Tuning

Using this custom-built model, we experimented on a wide number of factors. First of all, we discovered that small batch sizes were preferable to large batch sizes for this model. Although large batches converged to a better minima on the training data, it led to significant overfitting. Following suit with [research](https://arxiv.org/abs/1609.04836) recommending the use of small batch sizes, we decided to use batches of 32-64 images instead of 256-512.

### Model 1

**Stochastic Gradient Descent** _(10 epochs, 5e-3 rate, 0.9 momentum, 0.4 dropout (2 dropout layers), 64 image batch size)_: 83.32% test accuracy

<img src="model1.png" width="300">

As seen, however, the validation accuracy does not decrease significantly after 3 epochs, indicating a level of overfitting on the training data. Although 83% is a good accuracy, it is likely this won't generalise well with similar pictures from different sets. This model is likely overfit on this dataset's colours, detail, etc. We decided to try more techniques to deal with this.

### Model 2

**Stochastic Gradient Descent** _(10 epochs, 1e-3 rate, 0.9 momentum, 0.3 dropout (3 dropout layers), 64 image batch size, random flips, grayscale)_

<img src="model2.png" width="300">

**Adaptive Momentum Estimation (Adam)** _(20 epochs, 1e-4 rate, 0.9 momentum, 0.3 dropout (3 dropout layers), 64 image batch size, random flips, grayscale, scheduler (1 patience, 0.1 factor) on validation accuracy)_: 81.89% test accuracy

<img src="model3.png" width="300">

Here, to deal with the overfitting, I introduced a third dropout layer (between the convolutional layer and the first dense layer), and introduced data augmentation. The first augmentation is random vertical and horizontal flipping of training data, effectively quadrupling the "amount" of training data to reduce overfitting. Secondly, I grayscaled the images to reduce overfitting on colour saturation, since some [experts](https://cs230.stanford.edu/projects_winter_2019/reports/15813329.pdf) claim colour holds no meaning in cancer detection.

I carried out an 10 epochs of SGD followed by 20 epochs of Adam gradient descent, since the Adam optimiser is great at finding a minima regardless of training rate. I also introduced a scheduler at this point to fine tune the model. From this, it is clear to see how the Adam optimiser tends to overfit on training data, which is a known criticism of this optimiser. As such, we realise SGD is the best fit for our model. This test also showed that grayscale doesn't significantly reduce test accuracy, while reducing overfitting (as seen on the first 10 epochs) using SGD.

### Model 3

**Stochastic Gradient Descent** _(30 epochs, 1e-3 rate, 0.92 momentum, 0.35 dropout (3 dropout layers), 32 image batch size, random flips, grayscale, scheduler (2 patience, 0.4 factor) on validation accuracy)_: 84.87% test accuracy

<img src="model4.png" width="300">

Giving us a fantastic, almost 85% accuracy, this model is incredibly accurate at finding the cancerous examples of patches, while not significantly overfitting on the training data. Trained in about 1 hour on a 1080Ti, this model is both quick to train and accurate to test on unseen data.

# Code

### Setup

In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision.transforms import RandomHorizontalFlip, RandomVerticalFlip, Grayscale
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import h5py

In [2]:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

### Data Importing

In [3]:
class Data(Dataset):
    
    def __init__(self, train=0, gray=True):
        
        # Load data
        self.train = (train == 0)
        if train == 0:
            self.x = self.load_h5_as_numpy('camelyonpatch_level_2_split_train_x.h5', 'x')
            self.y = self.load_h5_as_numpy('camelyonpatch_level_2_split_train_y.h5', 'y')[:, 0, 0, 0]
        elif train == 1:
            self.x = self.load_h5_as_numpy('camelyonpatch_level_2_split_test_x.h5', 'x')
            self.y = self.load_h5_as_numpy('camelyonpatch_level_2_split_test_y.h5', 'y')[:, 0, 0, 0]
        elif train == 2:
            self.x = self.load_h5_as_numpy('camelyonpatch_level_2_split_valid_x.h5', 'x')
            self.y = self.load_h5_as_numpy('camelyonpatch_level_2_split_valid_y.h5', 'y')[:, 0, 0, 0]
        self.gray = gray
            
        # Prepare transforms
        self.t1 = RandomHorizontalFlip(p=0.5)
        self.t2 = RandomVerticalFlip(p=0.5)
        self.g = Grayscale()
            
    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, idx):
        
        # Get images
        if torch.is_tensor(idx):
            idx = idx.tolist()
        if type(idx) is list:
            img = torch.from_numpy(self.x[idx].astype(np.float32)).permute(0, 3, 1, 2)/128 - 1
            cls = torch.from_numpy(self.y[idx])
        else:
            img = torch.from_numpy(self.x[idx].astype(np.float32)).permute(2, 0, 1)/128 - 1
            cls = self.y[idx]
            
        # Transforms
        if self.train:
            img = self.t1(self.t2(img))
        if self.gray:
            img = self.g(img)
        
        # Return
        return (img, cls)
    
    def load_h5_as_numpy(self, file_name, key):
        with h5py.File(file_name, 'r') as h5_file:
            data = h5_file[key][:]
        return data

### Models

In [4]:
class CustomModel(nn.Module):
    
    def __init__(self, dropout_rate=0.2, gray=False):
        
        super().__init__()
        
        first_deg = 1 if gray else 3
        
        self.conv = nn.Sequential(
            
            nn.Conv2d(first_deg, 16, 3, padding='valid'),
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, 3, padding='valid'),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(2),    

            nn.Conv2d(32, 64, 3, padding='valid'),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding='valid'),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Flatten()
        )
        
        self.drop = nn.Dropout(dropout_rate)
        
        self.lin1 = nn.Sequential(
            nn.Linear(4096, 256),
            nn.ReLU()
        )
        
        self.lin2 = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU()
        )
        
        self.lin3 = nn.Linear(128, 2)
        
        self.train_model = nn.Sequential(
            self.conv,
            self.drop,
            self.lin1,
            self.drop,
            self.lin2,
            self.drop,
            self.lin3
        )
        
        self.test_model = nn.Sequential(
            self.conv,
            self.lin1,
            self.lin2,
            self.lin3
        )
    
    def forward(self, x):
        if self.training:
            return self.train_model(x)
        else:
            return self.test_model(x)
        

### Training / Testing Loop

In [5]:
class Log():
    
    def __init__(self):
        self.log_text = ""

    def log(self, text):
        self.log_text += (text + "\n")
        print(text)

    def get_log(self):
        return self.log_text

In [6]:
def training(model, loss_fn, optimiser, epoch, train_name, save=False, patience=1, factor=0.4):
    
    # Setup
    logger = Log()
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimiser, mode="max", patience=patience, factor=factor, verbose=True)
    train_loss = np.zeros(epoch)
    valid_loss = np.zeros(epoch)
    
    # Main epoch loop
    for i in range(epoch):

        # Training
        model.train()
        for inputs, labels in tqdm(train_dataloader, mininterval=1):
            y_pred = model(inputs.to(dev))
            loss = loss_fn(y_pred, labels.to(dev))
            train_loss[i] += loss.item()
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

        # Validating
        model.eval()
        acc = 0
        loss = 0
        for inputs, labels in tqdm(val_dataloader, mininterval=1):
            y_pred = model(inputs.to(dev))
            acc += (torch.argmax(y_pred, 1) == labels.to(dev)).float().sum()
            valid_loss[i] += loss_fn(y_pred, labels.to(dev)).item()
        acc = 100 * float(acc) / len(val_dataloader.dataset)
        logger.log(f"Epoch {i+1}: validation accuracy {round(acc, 2)}")
        scheduler.step(acc)
        
    # Normalise loss data
    train_loss = 100 * train_loss / len(train_dataloader)
    valid_loss = 100 * valid_loss / len(val_dataloader)
        
    # Create the loss plot
    epochs = np.arange(1, epoch+1)
    plt.plot(epochs, train_loss)
    plt.plot(epochs, valid_loss)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['Training Loss','Validation Loss'])
    
    # Save data
    if save:
        torch.save(model.state_dict(), f'{train_name}.model')
        plt.savefig(f'{train_name}_train.png')
        with open(f'{train_name}_train.txt', "w") as text_file:
            text_file.write(logger.get_log())

In [7]:
def testing(model, test_name, save=False):
    
    # Setup
    logger = Log()
    
    # Testing
    model.eval()
    acc = 0
    for inputs, labels in tqdm(test_dataloader, mininterval=1):
        y_pred = model(inputs.to(dev))
        acc += (torch.argmax(y_pred, 1) == labels.to(dev)).float().sum()
    acc = 100 * float(acc) / len(test_dataloader.dataset)
    logger.log(f"Test accuracy {round(acc, 2)}")
    
    # Saving
    if save:
        with open(f'{test_name}_test.txt', "w") as text_file:
            text_file.write(logger.get_log())

### Running

In [8]:
# Image settings
batch_size = 32
gray = True

# Model settings
dropout_rate = 0.4

# Training settings
lr = 3
momentum = 0.93
epochs = 40

# Scheduler settings
patience = 2
factor = 0.4

# Name
name = f"{epochs}_SGD_{lr}_{momentum}_{dropout_rate}_{batch_size}_d_t{'g' if gray else ''}"

In [9]:
train_dataloader = DataLoader(Data(train=0, gray=gray), batch_size=batch_size)
test_dataloader = DataLoader(Data(train=1, gray=gray), batch_size=batch_size)
val_dataloader = DataLoader(Data(train=2, gray=gray), batch_size=batch_size)

In [ ]:
model = CustomModel(dropout_rate=dropout_rate, gray=gray).to(dev)
loss_fn = nn.CrossEntropyLoss()

optimiser = torch.optim.SGD(model.parameters(), lr=10**(-lr), momentum=momentum)
training(model, loss_fn, optimiser, epochs, name, save=True, patience=patience, factor=factor)

In [ ]:
testing(model, name, save=True)